# Import Libraries

In [32]:
import numpy as np
import pandas as pd
import fairlearn.moments as moments
import fairlearn.classred as red
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Define Regressor Class

# Get Files and Inputs

In [14]:
import os
print(os.listdir('../input/fair-classification'))

['test_no_income.csv', 'test_sample0.csv', 'train.csv']


In [44]:
df = pd.read_csv('../input/fair-classification/train.csv')
submission_df = pd.read_csv('../input/fair-classification/test_no_income.csv')

In [45]:
df.columns

Index(['Age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital gain', 'capital loss', 'hours per week', 'native-country',
       'income'],
      dtype='object')

In [46]:
submission_df.shape

(8997, 14)

In [53]:
from sklearn.preprocessing import StandardScaler
norm_cols = ['Age', 'fnlwgt', 'education-num', 'capital gain',\
             'capital loss', 'hours per week']

scaler = StandardScaler()

df_categ = df.drop(norm_cols, axis=1)
submission_df_categ = submission_df.drop(norm_cols, axis=1)

df_norm = df[norm_cols]
submission_df_norm = submission_df[norm_cols]

df_normed = pd.DataFrame(scaler.fit_transform(df_norm), columns=norm_cols)
submission_df_normed = pd.DataFrame(scaler.fit_transform(submission_df_norm), columns=norm_cols)

df_normalized = pd.concat([df_categ, df_normed])
submission_df_normalized = pd.concat([submission_df_categ, submission_df_normed])

df_normed.head()

C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\justi\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1

,Age,fnlwgt,education-num,capital gain,capital loss,hours per week
0,0.383413,-0.857960,0.760842,-0.145449,-0.218389,0.614007
1,1.327465,2.706311,-0.415191,-0.145449,-0.218389,-0.029816
2,-1.286833,2.069482,-0.023180,-0.145449,-0.218389,-1.317462
3,-1.286833,0.039775,-0.415191,-0.145449,-0.218389,-0.029816
4,1.327465,-0.626370,1.936875,-0.145449,-0.218389,-0.432205


In [18]:
# encode string data
df = pd.get_dummies(df)
submission_df = pd.get_dummies(submission_df)

In [19]:
df.head()

,Age,fnlwgt,education-num,race,gender,capital gain,capital loss,hours per week,income,workclass_?,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,44,100479,12,0,0,0,0,48,0,0,...,0,0,0,0,0,0,0,1,0,0
1,57,477867,9,0,0,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
2,21,410439,10,0,0,0,0,24,0,0,...,0,0,0,0,0,0,0,1,0,0
3,21,195532,9,0,1,0,0,40,0,0,...,0,0,0,0,0,0,0,1,0,0
4,57,125000,15,0,0,0,0,35,1,0,...,0,0,0,0,0,0,0,1,0,0


In [20]:
missing_cols = set(df.columns) - set(submission_df.columns)

for c in missing_cols:
    submission_df[c] = 0
    
# Ensure Ordering matches
submission_df = submission_df[df.columns].drop('income', axis=1)

In [21]:
df.columns

Index(['Age', 'fnlwgt', 'education-num', 'race', 'gender', 'capital gain',
       'capital loss', 'hours per week', 'income', 'workclass_?',
       ...
       'native-country_Portugal', 'native-country_Puerto-Rico',
       'native-country_Scotland', 'native-country_South',
       'native-country_Taiwan', 'native-country_Thailand',
       'native-country_Trinadad&Tobago', 'native-country_United-States',
       'native-country_Vietnam', 'native-country_Yugoslavia'],
      dtype='object', length=103)

# Organize data into training, testing, features(X), sensitive attributes(A), targets(y)

In [24]:
train_df, test_df = train_test_split(df, test_size=0.25)
X_train, A_train, y_train = train_df.drop(['income','gender'], axis=1), train_df['gender'], train_df['income']
X_test, A_test, y_test = test_df.drop(['income','gender'], axis=1), test_df['gender'], test_df['income']

In [25]:
submission_df.shape[1], X_test.shape[1]

(102, 101)

# Using fairlearn to find best model

In [40]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

class LogisticRegressionLearner():
    def __init__(self):
        self.model = None
    def fit(self, X, Y, W):
        self.model = LogisticRegression(C=0.1, solver='liblinear', tol=1e-4, penalty='l1', verbose=1, max_iter=100000)
        self.model.fit(X, Y, sample_weight=W)

    def predict(self, X):
        return self.model.predict(X)


learner = LogisticRegressionLearner()
res_tuple = red.expgrad(X_train, A_train, y_train, learner,
                        cons=moments.DP(), eps=1000)
res = res_tuple._asdict()
best_classifier = res['best_classifier']

# Make Predictions
predictions = best_classifier(X_test)

def indicator(predictions):
    predictions[predictions < 0.5] = 0
    predictions[predictions >= 0.5] = 1
    return predictions

predictions = indicator(predictions)
accuracy = accuracy_score(y_test, predictions)
print('Accuracy: ', round(accuracy, 2), '%.')

# X_pred includes income columns that is predicted by the model
X_pred = test_df.drop('income', axis=1)
X_pred['income'] = predictions

A0 = len(X_pred.loc[X_pred['gender'] == 0])
A1 = len(X_pred.loc[X_pred['gender'] == 1])

Y1A0 = len(X_pred.loc[(X_pred['income'] == 1) & (X_pred['gender'] == 0)])
Y1A1 = len(X_pred.loc[(X_pred['income'] == 1) & (X_pred['gender'] == 1)])

Th = 0.1
DDP = np.abs((Y1A0 / A0) - (Y1A1 / A1))
bias = (7**(DDP - Th) - 1) if DDP > Th else 0
score = accuracy - bias
print(f'DDP: {DDP}\nAccuracy: {accuracy}\nBias: {bias}\nScore: {score}')

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]Accuracy:  0.86 %.
DDP: 0.001155497363865561
Accuracy: 0.8572857619206402
Bias: 0
Score: 0.8572857619206402


# Submission

In [ ]:
submission_df_ = submission_df.drop(['gender'], axis=1)
submission_predictions = indicator(best_classifier(submission_df_))

In [ ]:
import datetime

file_name = '../output/test_sample_' + str(datetime.datetime.now()).replace(' ', '').replace(':','') + '.csv'
file_name

In [ ]:
submission = pd.Series(submission_predictions, dtype=np.int64)

In [ ]:
submission.to_csv(file_name, index_label='Id', header=['income'], index=True)